# Análisis de ventas de productos digitales simuladas 

## Fase 3. Insight y segmentación 
El objetivo de esta fase es profundizar en patrones específicos y segmentar para estrategias dirigidas, se dividirá en el siguiente esquema:

```mermaid
    graph TD
    A[1 <br> Segmentación simple]
    B[2 <br> Hallazgos y recomendaciones]
    
    A-->B

    style A fill: #939a3f
    style B fill: #818555

```

1. **Segmetación simple:** Agrupar clientes por valor, frecuencia o preferencias.
    - **Ánálisis de Clientes ballena**
        1. Clientes VIP
        2. Caracterización 
        3. Diagnostico de riesgos 
        4. Estrategia de retención
2. **Hallazgos y recomendaciones:** Sintetizar insights clave y proponer acciones estratégicas



In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('C:/Users/nat27/Desktop/Desktop/Proyectos/CienciaDatos/digital-sales-analytics/data/clean/digital_products_sales_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          8000 non-null   int64  
 1   customer_id       8000 non-null   int64  
 2   product_name      8000 non-null   object 
 3   category          8000 non-null   object 
 4   price_usd         8000 non-null   float64
 5   quantity          8000 non-null   int64  
 6   discount_rate     8000 non-null   float64
 7   gross_amount_usd  8000 non-null   float64
 8   net_revenue_usd   8000 non-null   float64
 9   purchase_dt       8000 non-null   object 
 10  region            8000 non-null   object 
 11  channel           8000 non-null   object 
 12  payment_method    8000 non-null   object 
 13  order_date        8000 non-null   object 
 14  year              8000 non-null   int64  
 15  month             8000 non-null   int64  
 16  day               8000 non-null   int64  


## 1. Clientes VIP 

In [18]:
# 1. Calcular el valor total de cada cliente

customer_value = df.groupby('customer_id').agg({
    'net_revenue_usd': 'sum',
    'order_id': 'count'
}).rename(columns={
    'net_revenue_usd': 'total_spent',
    'order_id': 'total_orders'
}).sort_values('total_spent', ascending=False)

# 2. Calcular porcentaje acumulado de ingresos (Pareto)

total_revenue = customer_value['total_spent'].sum()
customer_value['cumulative_revenue'] = customer_value['total_spent'].cumsum()
customer_value['cumulative_pct'] = (customer_value['cumulative_revenue'] / total_revenue) * 100
customer_value['revenue_pct'] = (customer_value['total_spent'] / total_revenue) * 100

# 3. Identificar clientes que representan el 80% de ingresos (Regla de Pareto)

pareto_80_customers = customer_value[customer_value['cumulative_pct'] <= 80].index
n_pareto_80 = len(pareto_80_customers)
pct_pareto_80 = (n_pareto_80 / len(customer_value)) * 100

# 4. Identificar Top 5% y Top 10% de clientes

top_5pct_threshold = int(len(customer_value) * 0.05)
top_10pct_threshold = int(len(customer_value) * 0.10)

top_5pct_customers = customer_value.head(top_5pct_threshold).index
top_10pct_customers = customer_value.head(top_10pct_threshold).index

# 5. Calcular ingresos por cada grupo

revenue_pareto_80 = customer_value.loc[pareto_80_customers, 'total_spent'].sum()
revenue_top_5pct = customer_value.loc[top_5pct_customers, 'total_spent'].sum()
revenue_top_10pct = customer_value.loc[top_10pct_customers, 'total_spent'].sum()

# 6. Resumen de identificación de VIP
# =============================
print("\nResumen de identificación de VIP")
print(f"\nTotal de clientes únicos: {len(customer_value):,}")
print(f"Ingresos totales: ${total_revenue:,.2f}")
print()

print("OPCIÓN A: Regla de Pareto (80/20)")
print(f"  • Clientes que generan el 80% de ingresos: {n_pareto_80} ({pct_pareto_80:.1f}%)")
print(f"  • Ingresos generados: ${revenue_pareto_80:,.2f} ({(revenue_pareto_80/total_revenue)*100:.1f}%)")
print()

print("OPCIÓN B: Top 5%")
print(f"  • Número de clientes: {top_5pct_threshold}")
print(f"  • Ingresos generados: ${revenue_top_5pct:,.2f} ({(revenue_top_5pct/total_revenue)*100:.1f}%)")
print()

print("OPCIÓN C: Top 10%")
print(f"  • Número de clientes: {top_10pct_threshold}")
print(f"  • Ingresos generados: ${revenue_top_10pct:,.2f} ({(revenue_top_10pct/total_revenue)*100:.1f}%)")
print()




Resumen de identificación de VIP

Total de clientes únicos: 995
Ingresos totales: $374,647.04

OPCIÓN A: Regla de Pareto (80/20)
  • Clientes que generan el 80% de ingresos: 88 (8.8%)
  • Ingresos generados: $299,648.61 (80.0%)

OPCIÓN B: Top 5%
  • Número de clientes: 49
  • Ingresos generados: $275,719.58 (73.6%)

OPCIÓN C: Top 10%
  • Número de clientes: 99
  • Ingresos generados: $304,160.63 (81.2%)

